# COVID-19 Predictions with ARIMA model

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import random
import math
import time
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import operator 
from pmdarima.arima import auto_arima
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime
from datetime import date
import operator
from matplotlib import pyplot

In [2]:
confirmed_cases_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
coloco = ['Lat','Long']
confirmed_cases_global.drop(coloco, inplace=True, axis=1)
confirmed_cases_global=confirmed_cases_global[confirmed_cases_global['Province/State']!='Recovered']
confirmed_cases_global=confirmed_cases_global[confirmed_cases_global['Province/State']!='Diamond Princess']
confirmed_cases_global.head()


,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,16509,17267,18054,18969,19551,20342,20917,21459,22142,22890
1,NaN,Albania,0,0,0,0,0,0,0,0,...,1164,1184,1197,1212,1232,1246,1263,1299,1341,1385
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,9626,9733,9831,9935,10050,10154,10265,10382,10484,10589
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,844,851,852,852,852,852,852,852,852,852
4,NaN,Angola,0,0,0,0,0,0,0,0,...,86,86,86,86,88,91,92,96,113,118


In [3]:
#create a row for Canada
confirmed_cases_Canada=confirmed_cases_global[confirmed_cases_global['Country/Region']=='Canada']
confirmed_cases_Canada=confirmed_cases_Canada.groupby(['Country/Region']).sum().reset_index()
confirmed_cases_Canada.insert(0, 'Province/State', float('NaN'))
confirmed_cases_Canada

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20
0,NaN,Canada,0,0,0,0,1,1,2,2,...,93960,94641,95269,95947,96475,97178,97779,98241,98720,99159


In [4]:
confirmed_cases_US = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
confirmed_cases_US.rename(columns={'Province_State':'Province/State', 'Country_Region':'Country/Region', 'Long_':'Long' }, inplace=True)
coloc = ['UID', 'iso2','iso3' ,'code3', 'FIPS','Admin2','Combined_Key','Lat','Long']
confirmed_cases_US.drop(coloc, inplace=True, axis=1)
confirmed_cases_US.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20
0,American Samoa,US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guam,US,0,0,0,0,0,0,0,0,...,175,177,179,179,179,179,179,180,180,183
2,Northern Mariana Islands,US,0,0,0,0,0,0,0,0,...,23,24,26,26,26,27,28,30,30,30
3,Puerto Rico,US,0,0,0,0,0,0,0,0,...,3935,4023,4508,4620,4915,4985,5046,5185,5329,5352
4,Virgin Islands,US,0,0,0,0,0,0,0,0,...,70,70,71,71,71,71,71,71,72,72


In [5]:
confirmed_cases_US=confirmed_cases_US.groupby(['Province/State','Country/Region']).sum().reset_index()

In [6]:
deaths_cases_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
colo = ['Lat','Long']
deaths_cases_global=deaths_cases_global[deaths_cases_global['Province/State']!='Recovered']
deaths_cases_global=deaths_cases_global[deaths_cases_global['Province/State']!='Diamond Princess']
deaths_cases_global.drop(colo, inplace=True, axis=1)
deaths_cases_global.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,270,294,300,309,327,357,369,384,405,426
1,NaN,Albania,0,0,0,0,0,0,0,0,...,33,33,33,33,34,34,34,34,34,35
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,667,673,681,690,698,707,715,724,732,741
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,51,51,51,51,51,51,51,51,51,51
4,NaN,Angola,0,0,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,4,5


In [7]:
deaths_cases_Canada=deaths_cases_global[deaths_cases_global['Country/Region']=='Canada']
deaths_cases_Canada=deaths_cases_Canada.groupby(['Country/Region']).sum().reset_index()
deaths_cases_Canada.insert(0, 'Province/State', float('NaN'))
deaths_cases_Canada

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20
0,NaN,Canada,0,0,0,0,0,0,0,0,...,7477,7578,7716,7777,7849,7876,7909,7969,8037,8070


In [8]:
deaths_cases_US = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
deaths_cases_US.rename(columns={'Province_State':'Province/State', 'Country_Region':'Country/Region','Long_':'Long' }, inplace=True)
col = ['UID', 'iso2','iso3' ,'code3', 'FIPS','Admin2','Combined_Key', 'Population','Long','Lat']
deaths_cases_US.drop(col, inplace=True, axis=1)
deaths_cases_US.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20
0,American Samoa,US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guam,US,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
2,Northern Mariana Islands,US,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
3,Puerto Rico,US,0,0,0,0,0,0,0,0,...,138,140,140,141,142,142,142,142,143,144
4,Virgin Islands,US,0,0,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6


In [9]:
deaths_cases_US = deaths_cases_US.groupby(['Province/State','Country/Region']).sum().reset_index()


In [10]:
confirmed_cases= pd.concat([confirmed_cases_global, confirmed_cases_US,confirmed_cases_Canada], axis=0, sort=False)
confirmed_cases.to_csv('confirmed_cases_total.csv', sep=',',index = False)

In [11]:
deaths_cases = pd.concat([deaths_cases_global, deaths_cases_US,deaths_cases_Canada], axis=0, sort=False)
deaths_cases.to_csv('deaths_cases_total.csv', sep=',',index = False)

In [12]:
recoveries_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
coloco = ['Lat','Long']
recoveries_cases.drop(coloco, inplace=True, axis=1)
recoveries_cases.head()
recoveries_cases.to_csv('recoveries_cases_total.csv', sep=',',index = False)

In [13]:
colnames = confirmed_cases.columns.values

In [14]:
colonames = recoveries_cases.columns.values

In [15]:
confirmed = confirmed_cases.loc[:, colnames[4]:colnames[-1]]
deaths = deaths_cases.loc[:, colnames[4]:colnames[-1]]
recoveries = recoveries_cases.loc[:, colonames[4]:colonames[-1]]

In [16]:
time_date = confirmed.columns.values

In [17]:
all_days = np.array([i for i in range(len(time_date))]).reshape(-1, 1)

# Predictions for days ahead

In [18]:
pred_days_ahead = 2
#pred_days_ahead = 7
#pred_days_ahead = 30
predictions_ahead = np.array([i for i in range(len(time_date)+pred_days_ahead)]).reshape(-1, 1)

predictions_ahead_plus_day = np.array([i for i in range(len(time_date)+pred_days_ahead+1)]).reshape(-1, 1)

In [19]:
start = datetime.datetime.strptime(colnames[4], "%m/%d/%y")
end = datetime.datetime.strptime(colnames[-1], "%m/%d/%y")
predictions_ahead_dates = [start + datetime.timedelta(days=d) for d in range(len(predictions_ahead_plus_day))]

In [20]:
confirmed_cases = confirmed_cases[~confirmed_cases['Country/Region'].isin(['Tajikistan','Comoros'])]

#confirmed_cases = confirmed_cases[confirmed_cases['Country/Region'].isin(['Tajikistan'])]
confirmed_cases

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,16509,17267,18054,18969,19551,20342,20917,21459,22142,22890
1,NaN,Albania,0,0,0,0,0,0,0,0,...,1164,1184,1197,1212,1232,1246,1263,1299,1341,1385
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,9626,9733,9831,9935,10050,10154,10265,10382,10484,10589
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,844,851,852,852,852,852,852,852,852,852
4,NaN,Angola,0,0,0,0,0,0,0,0,...,86,86,86,86,88,91,92,96,113,118
5,NaN,Antigua and Barbuda,0,0,0,0,0,0,0,0,...,26,26,26,26,26,26,26,26,26,26
6,NaN,Argentina,0,0,0,0,0,0,0,0,...,18319,19268,20197,21037,22020,22794,23620,24761,25987,27373
7,NaN,Armenia,0,0,0,0,0,0,0,0,...,10009,10524,11221,11817,12364,13130,13325,13675,14103,14669
8,Australian Capital Territory,Australia,0,0,0,0,0,0,0,0,...,107,107,107,107,108,108,108,108,108,108
9,New South Wales,Australia,0,0,0,0,3,4,4,4,...,3104,3106,3110,3110,3109,3112,3114,3117,3117,3115


In [21]:
countries=confirmed_cases['Country/Region'].tolist()
province=confirmed_cases['Province/State'].tolist()

In [22]:
prediction_models = dict()
prediction_models_confirmed = {'SVR': 0}
prediction_models_deaths = {'SVR': 0}
prediction_models_recoveries = {'SVR': 0}

In [23]:
# Prediction Models

from sklearn import model_selection
from sklearn.svm import SVR
def SVM_regression(train_x, train_y, test_x, test_y, X_test,subpopulation,X_train, Y_train):
    
    # Tune parameters to find optimal ones
   # c = [0.01, 0.1, 1]
   # gamma = [0.01, 0.1, 1]
   #epsilon = [0.01, 0.1, 1]
   #shrinking = [True, False]
    #degree = [3, 4, 5, 6]
  #grid_parameters = {'C': c, 'gamma' : gamma ,'epsilon': epsilon, 'shrinking' : shrinking, 'degree':degree} 
   #regressor = RandomizedSearchCV(SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1, C=0.1), grid_parameters, scoring='neg_mean_squared_error', 
    #                               cv=5)
    
    if (subpopulation == 'confirmed'):
         degree = 3
    elif (subpopulation == 'deaths'):
        degree = 4
    else :
        degree = 5


    regressor = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=degree, C=0.1)
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['SVR'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['SVR'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['SVR'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions) 
    

In [24]:
from sklearn.ensemble import RandomForestRegressor
def RF_regression(train_x, train_y, test_x, test_y,X_test,subpopulation,X_train, Y_train):
    
    # Tune parameters to find optimal ones
    n_estimators = [100, 150, 300]
    grid_parameters = {'n_estimators': n_estimators}
#    regressor = RandomizedSearchCV(RandomForestRegressor(), grid_parameters, scoring='neg_mean_squared_error', 
#                                   cv=3)
    
    regressor = RandomForestRegressor(n_estimators = 200, random_state = 42) 
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions)


In [25]:
# Gradient Boosted Regression Trees (GBRT) 
from sklearn.ensemble import GradientBoostingRegressor

def Grad_Boost_regression(train_x, train_y, test_x, test_y,X_test,subpopulation, X_train, Y_train):
   
    # Tune parameters to find optimal ones
    n_estimators = [100, 150, 300]
    loss = ['ls','lad','huber']
    grid_parameters = {'n_estimators': n_estimators}
  #  regressor = RandomizedSearchCV(GradientBoostingRegressor(learning_rate=0.1), grid_parameters, scoring='neg_mean_squared_error', 
 #                                  cv=3)
    
    regressor =  GradientBoostingRegressor(n_estimators=500, learning_rate=0.1,
                                            max_depth=1, random_state=0, loss='huber')
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions)



In [26]:
#check if a timeseries is stationary
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()     
    rolstd = timeseries.rolling(12).std()
    
    #Perform Dickey-Fuller test:
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    return dfoutput
    

In [27]:
def pred_with_arima(timeseries,n_days_ahead):
    
    #find the optimal d 
    diff=timeseries-timeseries.shift(1)
    for d in range(1,10):
        if (test_stationarity(diff.dropna())['p-value']<0.1):
            break
        else: 
            d=d+1 
            diff=diff-diff.shift(1)
            
    #use the arima model to make predictions, use original timeseries with optimal d:
    model = pm.auto_arima(timeseries, start_p=0, start_q=0,
                             max_p=4, max_q=4,
                              seasonal=False,
                             d=d, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=False)
    #print(model)
    forecasts, conf_int =model.predict(n_days_ahead,return_conf_int=True ,alpha=0.05)
    
    return forecasts , conf_int  # returns a tuple, array of forecasts and all conf int

In [28]:
def Arima_Analysis(region,pred_days_ahead):
   #create a dataframe for timeseries analysis
    date_rng = pd.date_range(start=colnames[4], end=colnames[-1], freq='D') #creates the days
    df = pd.DataFrame(date_rng, columns=['date'])
    df['cases']=np.transpose(region) #this is a dataframe with dates and cases.
    df = df.set_index('date')
    df=df[df['cases'] != 0] #remove zeroes, they do not help for analysis. 
    if (len(df)>1):
        if(df['cases'][-1]-df['cases'][0]>5):
            pred_ts,conf_int=pred_with_arima(df['cases'],pred_days_ahead) #this is an array converted to a list
        else:
            pred_ts=[df['cases'][-1]+1]*pred_days_ahead
            conf_int=[np.array([df['cases'][-1],df['cases'][-1]+2])]*pred_days_ahead
    else:
        pred_ts=np.zeros(pred_days_ahead)
        conf_int=[np.zeros(pred_days_ahead)] * pred_days_ahead
    return pred_ts, conf_int

In [29]:
import operator
def subpopulation_Analysis(cases,country,province, all_days,predictions_ahead,subpopulation, pred_days_ahead):
    
    pred  = list()
    if pd.isnull(province):
        region = cases.loc[(cases['Country/Region'] == country) & (pd.isnull(cases['Province/State']))]
    else :
        region = cases.loc[(cases['Country/Region'] == country) & (cases['Province/State'] == province)]
    print(region)
    
    reg =  np.array(region.loc[:, colnames[4]:colnames[-1]])
    region =  np.array(region.loc[:, colnames[4]:colnames[-1]]).reshape(-1, 1)
        
    #train_x, test_x, train_y, test_y = model_selection.train_test_split(all_days,region,test_size = 0.04,random_state = 42,shuffle=False)
    
    #pred_svm =  SVM_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_svm)
    
    #TODO: a for loop
    daily_increase_threshold1=np.absolute((reg[-1][-1]-reg[-1][-2])/reg[-1][-2])*100
    daily_increase_threshold2=np.absolute((reg[-1][-2]-reg[-1][-3])/reg[-1][-3])*100
    daily_increase_threshold3=np.absolute((reg[-1][-3]-reg[-1][-4])/reg[-1][-4])*100
    daily_increase_threshold4=np.absolute((reg[-1][-4]-reg[-1][-5])/reg[-1][-5])*100
    daily_increase_bound=1000
    daily_increase_threshold = max(daily_increase_threshold1,daily_increase_threshold2,daily_increase_threshold3,daily_increase_threshold4)
    
    if (daily_increase_threshold>daily_increase_bound):
        best_pred_ahead = 0  
        region = region
        best_pred = 0
        best_conf_int = np.zeros(2)
        reject_country = 1
        return(best_pred_ahead, region, best_pred, best_conf_int, reject_country)
     
    
    pred_arima, conf_int=Arima_Analysis(reg,pred_days_ahead)
    #pred.append(pred_svm)
    #pred_RF = RF_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_RF)

    
   # pred_gb =  Grad_Boost_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
   # pred.append(pred_gb)


    if (subpopulation == 'confirmed'): 
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_confirmed.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
    elif (subpopulation == 'deaths'):
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_deaths.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
    else :
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_recoveries.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
        

    best_pred_ahead = best_pred[-1]
    best_conf_int=conf_int[-1]
   
    best_conf_int[0]=max(reg[-1][-1], best_conf_int[0])
    if (best_conf_int[0]>best_conf_int[1]):
        best_conf_int[1]=2*reg[-1][-1]

    if(best_pred_ahead<reg[-1][-1]):
        best_pred_ahead= reg[-1][-1]+pred_days_ahead
    
    reject_country = 0
    
    return best_pred_ahead, region, best_pred, best_conf_int, reject_country

In [30]:
pred_file = str(2)+"day_prediction_" +  date.today().strftime('%Y-%m-%d') + "_arima.csv"
#pred_file = str(7)+"day_prediction_" + date.today().strftime('%Y-%m-%d') + "_arima.csv"
#pred_file = str(30)+"day_prediction_" + date.today().strftime('%Y-%m-%d') + "_arima.csv"
pred_file

'2day_prediction_2020-06-12_arima.csv'

In [31]:
columns = ['Province/State','Country','Target/Date','N','low95N','high95N',
          'R','low95R','high95R','D','low95D','high95D','T','low95T','high95T',
           'M','low95M','high95M','C','low95C','high95C']

            

k = 0
for co in range(len(countries)):
    
    ignore_recoveries = 0    
    
    total_predictions = pd.DataFrame(columns = columns)
    
    k = k + 1
       
    best_pred_confirmed, region_con, best_pred_con, conf_int_con, reject_country = subpopulation_Analysis(confirmed_cases,countries[co],province[co],
                                                 all_days,predictions_ahead,"confirmed",pred_days_ahead)
    
    if (reject_country==1):
        print('rejected_conf')
        continue
        
    best_pred_deaths, region_d, best_pred_d, conf_int_d, reject_country = subpopulation_Analysis(deaths_cases,countries[co],province[co],
                                              all_days,predictions_ahead,"deaths",pred_days_ahead)
    if (reject_country==1):
        print('rejected_d')
        continue
        
    if ((countries[co] == 'US' or countries[co] == 'Canada' ) & (pd.notnull(province[co]))):
        ignore_recoveries=1   
    else:
        best_pred_recoveries, region_r, best_pred_r, conf_int_r, reject_country = subpopulation_Analysis(recoveries_cases,countries[co],province[co],
                                                  all_days,predictions_ahead,"recoveries",pred_days_ahead)
    
    if (reject_country==1):
        print('rejected_r')
        continue
    
    # Mortality rate
    if (conf_int_con[0]==0):
        mort_pred=0
        mort_pred_low=0
        mort_pred_up=0
    else:
        mort_pred  = best_pred_deaths/best_pred_confirmed
        mort_pred_low= conf_int_d[0]/conf_int_con[1]
        mort_pred_up=conf_int_d[1]/conf_int_con[0]
    
    #fix province
    province_name=''
    if (pd.notnull(province[co])) :
        province_name=province[co]
    
    if (ignore_recoveries==0):
        R_value=int(best_pred_recoveries) 
        low_R=int(conf_int_r[0])
        high_R=int(conf_int_r[1])
    else:
        R_value=''
        low_R=''
        high_R=''
        
    #Critical Cases:
    #Assume that 5% of all cases are critical and need hospitalization:
    serious_percentage=0.05
    critical=serious_percentage*best_pred_confirmed
    low_critical=serious_percentage*conf_int_con[0]
    high_critical=serious_percentage*conf_int_con[1]
    
    total_predictions = total_predictions.append({'Province/State': province_name,
                                                'Country': countries[co],
                                                'Target/Date': predictions_ahead_dates[-1].strftime('%Y-%m-%d'), 
                                                'N': int(best_pred_confirmed) ,
                                                'low95N':int(conf_int_con[0]),
                                                'high95N': int(conf_int_con[1]),
                                                'R': R_value ,
                                                'low95R' : low_R,
                                                'high95R' : high_R,
                                                'D': int(best_pred_deaths) ,
                                                'low95D' : int(conf_int_d[0]),
                                                'high95D': int(conf_int_d[1]),
                                                'T': '',
                                                'low95T' : '' ,
                                                'high95T' : '',
                                                'M': mort_pred,
                                                'low95M': mort_pred_low,
                                                'high95M': mort_pred_up ,
                                                'C': int(critical) ,
                                                'low95C':  int(low_critical) ,
                                                'high95C': int(high_critical) },ignore_index = True)
    
    if (k==1):
        header = True
    else:
        header = False
    
    total_predictions.to_csv(pred_file, sep=',',index = False,  mode='a', header = header)
    

  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
0            NaN    Afghanistan        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  \
0        0        0        0  ...   16509   17267   18054   18969   19551   

   6/7/20  6/8/20  6/9/20  6/10/20  6/11/20  
0   20342   20917   21459    22142    22890  

[1 rows x 144 columns]
Fit ARIMA: (0, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=1253.974, BIC=1259.320, Time=0.138 seconds
Fit ARIMA: (0, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=1217.613, BIC=1225.631, Time=0.177 seconds
Fit ARIMA: (0, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=1219.562, BIC=1230.253, Time=0.400 seconds
Fit ARIMA: (0, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=1217.901, BIC=1231.265, Time=0.424 seconds
Fit ARIMA: (0, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=1219.590, BIC=1235.627, Time=0.394 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=1235.292, BIC=1243.310,

Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=640.480, BIC=653.196, Time=0.313 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=641.726, BIC=656.986, Time=0.463 seconds
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=643.473, BIC=661.276, Time=0.511 seconds
Fit ARIMA: (3, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=640.481, BIC=653.198, Time=0.214 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=642.482, BIC=657.742, Time=0.213 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=643.625, BIC=661.428, Time=0.720 seconds
Fit ARIMA: (4, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=642.466, BIC=657.726, Time=0.246 seconds
Fit ARIMA: (4, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=643.354, BIC=661.157, Time=0.626 seconds
Total fit time: 5.142 seconds
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
1            NaN        Albania        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  6/2/20

Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=550.167, BIC=557.700, Time=0.100 seconds
Fit ARIMA: (0, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=525.514, BIC=535.557, Time=0.190 seconds
Fit ARIMA: (0, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=504.007, BIC=516.561, Time=0.328 seconds
Fit ARIMA: (0, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=494.904, BIC=509.969, Time=0.506 seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=501.876, BIC=509.408, Time=0.120 seconds
Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=492.041, BIC=502.085, Time=0.283 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=489.798, BIC=502.352, Time=0.317 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=490.119, BIC=505.184, Time=0.624 seconds
Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=489.205, BIC=506.781, Time=0.555 seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=489.976, BIC=500.019, Time=0.125 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (const

Fit ARIMA: (3, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=208.667, BIC=222.959, Time=0.565 seconds
Fit ARIMA: (3, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=209.500, BIC=226.174, Time=0.581 seconds
Fit ARIMA: (4, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=214.564, BIC=228.856, Time=0.115 seconds
Fit ARIMA: (4, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=209.862, BIC=226.536, Time=0.714 seconds
Total fit time: 7.758 seconds
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
3            NaN        Andorra        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  \
3        0        0        0  ...     733     735     738     741     741   

   6/7/20  6/8/20  6/9/20  6/10/20  6/11/20  
3     744     751     757      759      780  

[1 rows x 144 columns]
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=700.930, BIC=705.930, Time=0.021 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=702

Fit ARIMA: (0, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=248.723, BIC=256.189, Time=0.178 seconds
Fit ARIMA: (0, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=250.687, BIC=260.641, Time=0.455 seconds
Fit ARIMA: (0, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=252.472, BIC=264.916, Time=0.266 seconds
Fit ARIMA: (0, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=250.815, BIC=265.747, Time=0.739 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=285.443, BIC=292.909, Time=0.051 seconds
Fit ARIMA: (1, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=250.693, BIC=260.648, Time=0.402 seconds
Fit ARIMA: (1, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=251.415, BIC=263.858, Time=0.363 seconds
Fit ARIMA: (1, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=252.311, BIC=267.242, Time=0.801 seconds
Fit ARIMA: (1, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=248.432, BIC=265.852, Time=1.587 seconds
Fit ARIMA: (2, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=276.127, BIC=286.081, Time=0.225 seconds
Fit ARIMA: (2, 2, 1)x(0, 0, 0, 0) (const

...

Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=340.667, BIC=350.245, Time=0.109 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=342.584, BIC=354.557, Time=0.136 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=339.650, BIC=354.017, Time=0.513 seconds
Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=340.508, BIC=357.270, Time=0.165 seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=340.225, BIC=349.803, Time=0.067 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=342.225, BIC=354.197, Time=0.141 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=340.140, BIC=354.507, Time=0.462 seconds
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=340.201, BIC=356.962, Time=0.513 seconds
Fit ARIMA: (3, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=342.222, BIC=354.194, Time=0.092 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=344.225, BIC=358.592, Time=0.101 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (const

Fit ARIMA: (0, 3, 1)x(0, 0, 0, 0) (constant=True); AIC=1273.442, BIC=1281.040, Time=0.196 seconds
Fit ARIMA: (0, 3, 2)x(0, 0, 0, 0) (constant=True); AIC=1244.370, BIC=1254.501, Time=0.196 seconds
Fit ARIMA: (0, 3, 3)x(0, 0, 0, 0) (constant=True); AIC=1245.520, BIC=1258.183, Time=0.341 seconds
Fit ARIMA: (0, 3, 4)x(0, 0, 0, 0) (constant=True); AIC=1246.335, BIC=1261.530, Time=0.518 seconds
Fit ARIMA: (1, 3, 0)x(0, 0, 0, 0) (constant=True); AIC=1324.938, BIC=1332.536, Time=0.047 seconds
Fit ARIMA: (1, 3, 1)x(0, 0, 0, 0) (constant=True); AIC=1260.733, BIC=1270.863, Time=0.203 seconds
Fit ARIMA: (1, 3, 2)x(0, 0, 0, 0) (constant=True); AIC=1245.725, BIC=1258.388, Time=0.359 seconds
Fit ARIMA: (1, 3, 3)x(0, 0, 0, 0) (constant=True); AIC=1245.611, BIC=1260.807, Time=0.447 seconds
Fit ARIMA: (1, 3, 4)x(0, 0, 0, 0) (constant=True); AIC=1247.102, BIC=1264.830, Time=0.614 seconds
Fit ARIMA: (2, 3, 0)x(0, 0, 0, 0) (constant=True); AIC=1298.291, BIC=1308.421, Time=0.047 seconds
Fit ARIMA: (2, 3, 1)

Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=707.563, BIC=723.372, Time=0.373 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=709.024, BIC=727.467, Time=0.534 seconds
Fit ARIMA: (4, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=710.538, BIC=726.346, Time=0.195 seconds
Fit ARIMA: (4, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=709.523, BIC=727.966, Time=0.485 seconds
Total fit time: 4.699 seconds
   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
55  West Virginia             US        0        0        0        0        0   

    1/27/20  1/28/20  1/29/20  ...  6/2/20  6/3/20  6/4/20  6/5/20  6/6/20  \
55        0        0        0  ...    2056    2077    2102    2119    2136   

    6/7/20  6/8/20  6/9/20  6/10/20  6/11/20  
55    2144    2161    2169     2194     2217  

[1 rows x 144 columns]
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=782.095, BIC=786.980, Time=0.016 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); A

Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=519.656, BIC=529.379, Time=0.238 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=521.643, BIC=533.797, Time=0.295 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=524.303, BIC=538.887, Time=0.247 seconds
Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=526.301, BIC=543.317, Time=0.141 seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=521.178, BIC=530.901, Time=0.094 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=521.642, BIC=533.796, Time=0.349 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=523.416, BIC=538.001, Time=0.461 seconds
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=520.029, BIC=537.045, Time=0.535 seconds
Fit ARIMA: (3, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=522.675, BIC=534.829, Time=0.156 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=522.728, BIC=537.312, Time=0.367 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (const

Fit ARIMA: (0, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=943.735, BIC=953.866, Time=0.156 seconds
Fit ARIMA: (0, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=939.821, BIC=952.484, Time=0.200 seconds
Fit ARIMA: (0, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=937.461, BIC=952.657, Time=0.251 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=961.641, BIC=969.239, Time=0.078 seconds
Fit ARIMA: (1, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=944.313, BIC=954.443, Time=0.156 seconds
Fit ARIMA: (1, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=936.805, BIC=949.468, Time=0.187 seconds
Fit ARIMA: (1, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=936.852, BIC=952.048, Time=0.250 seconds
Fit ARIMA: (1, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=938.526, BIC=956.254, Time=0.449 seconds
Fit ARIMA: (2, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=951.264, BIC=961.394, Time=0.083 seconds
Fit ARIMA: (2, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=944.273, BIC=956.936, Time=0.191 seconds
Fit ARIMA: (2, 2, 2)x(0, 0, 0, 0) (const